In [1]:
import pickle
import numpy as np
from ex_2_3 import Node, load_params, load_sample, print_tree
from ex_2_3_tree_helper import Tree

In [2]:
t = Tree()

In [23]:
my_data_path = ''

with open(my_data_path + 'tree_with_CPD.pkl', 'rb') as handle:
    params = pickle.load(handle, encoding='latin1')

with open(my_data_path + 'tree_with_leaf_samples.pkl', 'rb') as handle:
    samples = pickle.load(handle, encoding='latin1')

In [24]:
#params

In [25]:
"""
    Construct a tree with parameters from the loaded parameter dict.
"""
#params_name = list(params.keys())[10]     
#param = params[params_name]
param = params
root = load_params(param)
K = len(root.cat[0])
"""
    Load a matching sample into the tree.
"""
#samples_name = params_name + '_sample_1'
#sample = samples[samples_name]
sample = samples
load_sample(root, sample)
"""
Print the tree (not very sophisticated). Structure: nodename_parentname
"""
#print(params_name)
print_tree(root)
"""
Print the tree with sample (not very sophisticated). Structure: nodename_parentname:sample
"""
print_tree(root, print_sample = True)

1 
2_pa1 3_pa1 4_pa1 
5_pa2 6_pa2 7_pa3 8_pa3 9_pa3 10_pa3 11_pa4 12_pa4 
13_pa7 14_pa7 15_pa7 16_pa7 17_pa8 18_pa8 19_pa8 20_pa8 21_pa9 22_pa10 23_pa10 24_pa11 25_pa12 
26_pa13 27_pa13 28_pa14 29_pa15 30_pa15 31_pa15 32_pa16 33_pa16 34_pa16 35_pa16 36_pa17 37_pa17 38_pa17 39_pa17 40_pa18 41_pa18 42_pa18 43_pa18 44_pa19 45_pa19 46_pa20 47_pa22 48_pa22 49_pa22 50_pa22 51_pa23 52_pa23 53_pa23 54_pa24 
1:Xv 
2_pa1:Xv 3_pa1:Xv 4_pa1:Xv 
5_pa2:0 6_pa2:1 7_pa3:Xv 8_pa3:Xv 9_pa3:Xv 10_pa3:Xv 11_pa4:Xv 12_pa4:Xv 
13_pa7:Xv 14_pa7:Xv 15_pa7:Xv 16_pa7:Xv 17_pa8:Xv 18_pa8:Xv 19_pa8:Xv 20_pa8:Xv 21_pa9:0 22_pa10:Xv 23_pa10:Xv 24_pa11:Xv 25_pa12:0 
26_pa13:0 27_pa13:1 28_pa14:1 29_pa15:0 30_pa15:1 31_pa15:1 32_pa16:1 33_pa16:1 34_pa16:1 35_pa16:1 36_pa17:0 37_pa17:0 38_pa17:1 39_pa17:1 40_pa18:0 41_pa18:1 42_pa18:1 43_pa18:1 44_pa19:0 45_pa19:1 46_pa20:0 47_pa22:1 48_pa22:1 49_pa22:0 50_pa22:0 51_pa23:0 52_pa23:0 53_pa23:1 54_pa24:0 


In [26]:
# dictionary to hash computed values of sub-problems (massive time saving for big trees)
sd_dict = {}

def sub_problem(node, i):
    #check if leaf node
    name = str(node)+'_'+ str(i)
    if name in sd_dict.keys(): return sd_dict[name]
    if len(node.descendants) == 0:
        if node.sample == i:
            sd_dict[name] = 1
            return 1
        else:
            sd_dict[name] = 0
            return 0
    else:
        P = 1
        #iterate over descendants of the node
        for child in node.descendants:
            S = 0
            #iterate for all possible values of the sample
            for j in range(K):
                S += child.cat[i][j]*sub_problem(child, j)
            P *= S
        sd_dict[name] = P
        return P
    
def sample(node):
    if len(node.descendants) == 0:
        return 'LN'
    cat = []
    for i in range(K):
        if node == root:
            cat.append(sub_problem(node,i)*node.cat[0][i])
        else:
            cat.append(sub_problem(node,i)*node.cat[int(node.ancestor.sample)][i])
    cat = [x / sum(cat) for x in cat]
    #print(cat)
    sample = list(np.random.dirichlet(cat))
    sample = sample.index(max(sample))
    #set the node value to new sample generated so that while sampling children we can use it from the tree itself
    node.sample = sample
    return sample

# Print Tree (Just a minor change to the provided code). Here we sample as we print.
def print_treeSample(root, print_sample = False):
    curr_layer = [root]
    while curr_layer != []:
        string = ''
        next_layer = []
        for elem in curr_layer:
            if elem.ancestor != None:
                string = string + '{0}_pa{1} '.format(elem.name, elem.ancestor.name) 
            else:
                string = string + elem.name  + ' ' 
            if (print_sample and elem.sample != None):
                string = string[:-1] + ':' + str(sample(elem))  + ' ' 
            for child in elem.descendants:
                next_layer.append(child)   
        print(string)
        curr_layer = next_layer

In [28]:
print_treeSample(root, True)

1:0 
2_pa1:1 3_pa1:1 4_pa1:0 
5_pa2:LN 6_pa2:LN 7_pa3:1 8_pa3:0 9_pa3:0 10_pa3:1 11_pa4:1 12_pa4:0 
13_pa7:1 14_pa7:0 15_pa7:1 16_pa7:1 17_pa8:1 18_pa8:0 19_pa8:0 20_pa8:0 21_pa9:LN 22_pa10:0 23_pa10:1 24_pa11:0 25_pa12:LN 
26_pa13:LN 27_pa13:LN 28_pa14:LN 29_pa15:LN 30_pa15:LN 31_pa15:LN 32_pa16:LN 33_pa16:LN 34_pa16:LN 35_pa16:LN 36_pa17:LN 37_pa17:LN 38_pa17:LN 39_pa17:LN 40_pa18:LN 41_pa18:LN 42_pa18:LN 43_pa18:LN 44_pa19:LN 45_pa19:LN 46_pa20:LN 47_pa22:LN 48_pa22:LN 49_pa22:LN 50_pa22:LN 51_pa23:LN 52_pa23:LN 53_pa23:LN 54_pa24:LN 
